
# DATABEHANDLING


### Starter med at indhente CSV-filer til DF's og lave disse til én stor

In [126]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import re
import itertools
import random
import threading
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import csv
import numpy as np
import re

In [317]:
pd.set_option('display.max_colwidth', None)
csv_files = ['rawdata/index_0_1000.csv', 'rawdata/index_1000_2000.csv', 'rawdata/index_2000_3000.csv', 'rawdata/index_3000_4000.csv', 'rawdata/index_4000_5000.csv', 'rawdata/index_5000_6000.csv', 'rawdata/index_6000_7000.csv','rawdata/index_7000_8000.csv', 'rawdata/index_8000_9000.csv', 'rawdata/index_9000_10000.csv']
dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=False)
combined_df.set_index('Unnamed: 0', inplace=True)
combined_df.index.name = None
combined_df

,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Toilet
"Mellemtoftevej 36,2600",3.995.000 kr.,"Hospitalsskolen , Børne og Ungdomspsykiatrisk Center",391 m,Ikke oplyst,Ikke oplyst,Rækkehus,Energimærke D,Høj,"24,6%",25‰.,...,Ejerudg.: 2.889 kr/md,5 værelser,Opført 1947,1 toilet,2600,https://www.boligsiden.dk/addresses/0a3f50a4-be85-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Strandpromenaden 21, 1. tv.,3000",6.095.000 kr.,Skolen i Bymidten,336 m,Ikke oplyst,Ikke oplyst,Ejerlejlighed,Energimærke A2015,Lav,"25,52%","28,5‰.",...,Ejerudg.: 6.056 kr/md,5 værelser,Opført 2020,2 toiletter,3000,https://www.boligsiden.dk/addresses/0d5cec39-0046-4f36-9850-8c2d08933fd5,Elevator: Nej,Altan: Nej,Terrasse: Nej,NaN
"Hjortdalvej 331,9460",3.995.000 kr.,Skovsgård Tranum skole,"5,24 km",6.9,3.6,Landejendom,Energimærke D,Meget lav,"25,7%","32,85‰.",...,Ejerudg.: 2.089 kr/md,5 værelser,Opført 1912,1 toilet,9460,https://www.boligsiden.dk/addresses/39ff92cd-f2b1-0991-e044-0003ba298018,NaN,NaN,NaN,NaN
"Lystgårdsparken 34,8300",2.798.000 kr.,Parkvejens Skole,"5,16 km",7,3.7,Villa,Energimærke B,Mellem,"25,1%","30,32‰.",...,Ejerudg.: 1.838 kr/md,4 værelser,Opført 2008,1 toilet,8300,https://www.boligsiden.dk/addresses/363cc2db-6e07-1eb8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Fuglsøvej 92,8420",6.000.000 kr.,Molsskolen,"3,74 km",7.4,3.7,Villa,Energimærke D,Høj,"25,9%","33,8‰.",...,Ejerudg.: 3.434 kr/md,12 værelser,Opført 1884,4 toiletter,8420,https://www.boligsiden.dk/addresses/0a3f50bf-6f17-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Enghavevej 4,9530",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/0a3f50c9-5dc4-32b8-e044-0003ba298018,fejl,fejl,fejl,fejl
"Søndergårdsparken 1,8660",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/2734ffdb-b506-4c83-9823-d3f41e7c24a7,fejl,fejl,fejl,fejl
"Højbyvej 16,4500",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/0a3f50ae-0819-32b8-e044-0003ba298018,fejl,fejl,fejl,fejl
"Vievang Alle 2A,2610",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,fejl,fejl,fejl,NaN,fejl,https://www.boligsiden.dk/addresses/0a3f50a6-c2ac-32b8-e044-0003ba298018,fejl,fejl,fejl,fejl


In [319]:
#Sorterer alt data med fejl i pris fra:
#Gemmer missing data i en dataframe
df_med_fejl=combined_df[pd.isna(combined_df['Price'])]
df_med_fejl


#Overskriver samlet dataframe, således at missing data (pga. fejl i kode) ikke optræder.
combined_df = combined_df.drop(df_med_fejl.index)
combined_df

,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Ejerudgift,Rooms,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Toilet
"Mellemtoftevej 36,2600",3.995.000 kr.,"Hospitalsskolen , Børne og Ungdomspsykiatrisk Center",391 m,Ikke oplyst,Ikke oplyst,Rækkehus,Energimærke D,Høj,"24,6%",25‰.,...,Ejerudg.: 2.889 kr/md,5 værelser,Opført 1947,1 toilet,2600,https://www.boligsiden.dk/addresses/0a3f50a4-be85-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Strandpromenaden 21, 1. tv.,3000",6.095.000 kr.,Skolen i Bymidten,336 m,Ikke oplyst,Ikke oplyst,Ejerlejlighed,Energimærke A2015,Lav,"25,52%","28,5‰.",...,Ejerudg.: 6.056 kr/md,5 værelser,Opført 2020,2 toiletter,3000,https://www.boligsiden.dk/addresses/0d5cec39-0046-4f36-9850-8c2d08933fd5,Elevator: Nej,Altan: Nej,Terrasse: Nej,NaN
"Hjortdalvej 331,9460",3.995.000 kr.,Skovsgård Tranum skole,"5,24 km",6.9,3.6,Landejendom,Energimærke D,Meget lav,"25,7%","32,85‰.",...,Ejerudg.: 2.089 kr/md,5 værelser,Opført 1912,1 toilet,9460,https://www.boligsiden.dk/addresses/39ff92cd-f2b1-0991-e044-0003ba298018,NaN,NaN,NaN,NaN
"Lystgårdsparken 34,8300",2.798.000 kr.,Parkvejens Skole,"5,16 km",7,3.7,Villa,Energimærke B,Mellem,"25,1%","30,32‰.",...,Ejerudg.: 1.838 kr/md,4 værelser,Opført 2008,1 toilet,8300,https://www.boligsiden.dk/addresses/363cc2db-6e07-1eb8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Fuglsøvej 92,8420",6.000.000 kr.,Molsskolen,"3,74 km",7.4,3.7,Villa,Energimærke D,Høj,"25,9%","33,8‰.",...,Ejerudg.: 3.434 kr/md,12 værelser,Opført 1884,4 toiletter,8420,https://www.boligsiden.dk/addresses/0a3f50bf-6f17-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Langelandsvej 23,6100",1.145.000 kr.,Starup-Øsby Skole og Børnehus,"1,14 km",6.4,3.8,Villa,Energimærke D,Høj,"26,3%","24,36‰.",...,Ejerudg.: 1.586 kr/md,5 værelser,Opført 1981,2 toiletter,6100,https://www.boligsiden.dk/addresses/0a3f50b6-c77b-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Nybovej 7,9640",995.000 kr.,Farsø Skole,373 m,6.9,3.7,Villa,Energimærke D,Mellem,"26,3%","22,82‰.",...,Ejerudg.: 979 kr/md,3 værelser,Opført 1954,1 toilet,9640,https://www.boligsiden.dk/addresses/0a3f50c7-2cfc-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Storskovvej 3,4700",2.995.000 kr.,"Fladsåskolen, afd. Korskilde","4,91 km",Ikke oplyst,Ikke oplyst,Villa,Energimærke A2010,Høj,25%,25‰.,...,Ejerudg.: 1.629 kr/md,5 værelser,Opført 1912,1 toilet,4700,https://www.boligsiden.dk/addresses/0a3f50af-9285-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN
"Gl Landevej 23,9690",725.000 kr.,Fjerritslev Skole,"5,62 km",6.4,3.6,Villa,Energimærke E,Lav,"25,7%","32,85‰.",...,Ejerudg.: 1.715 kr/md,5 værelser,Opført 1899,1 toilet,9690,https://www.boligsiden.dk/addresses/0a3f50c7-3af4-32b8-e044-0003ba298018,NaN,NaN,NaN,NaN


In [321]:
def categorize_boligtype(x):
    if 'Landejendom' in x:
        return 'Landejendom'
    elif 'Villalejlighed' in x:
        return 'Villalejlighed'
    elif 'Villa' in x:
        return 'Villa'
    elif 'Ejerlejlighed' in x:
        return 'Ejerlejlighed'
    elif 'Rækkehus' in x:
        return 'Rækkehus'
    else:
        return x

combined_df['Boligtype'] = combined_df['Boligtype'].apply(categorize_boligtype)
combined_df['Boligtype'] = combined_df['Boligtype'].astype('category')



In [323]:
#Kommuneskat
print(combined_df['Kommuneskat'].dtype)
combined_df['Kommuneskat'] = combined_df['Kommuneskat'].astype(str)
combined_df['Kommuneskat'] = combined_df['Kommuneskat'].str.replace(',', '.').str.replace('%', '') #komma til punktum og procenttegn væk
combined_df['Kommuneskat'] = combined_df['Kommuneskat'].astype(float)


print(combined_df['Ejerudgift'].dtype)
combined_df['Ejerudgift'] = combined_df['Ejerudgift'].astype(str)
combined_df['Ejerudgift'] = combined_df['Ejerudgift'].str.replace('Ejerudg.:', '').str.replace('kr/md', '') #komma til punktum og procenttegn væk
combined_df['Ejerudgift'] = combined_df['Ejerudgift'].astype(float)   #JEG HAR UDSKIFTET FEJL MED MISSING VALUES

print(combined_df['Grundskyldspromillen'].dtype)
combined_df['Grundskyldspromillen'] = combined_df['Grundskyldspromillen'].astype(str)
combined_df['Grundskyldspromillen'] = combined_df['Grundskyldspromillen'].str.replace(',', '.').str.replace('‰', '') #komma til punktum og procenttegn væk
combined_df['Grundskyldspromillen'] = combined_df['Grundskyldspromillen'].str.rstrip('.')
combined_df['Grundskyldspromillen'] = combined_df['Grundskyldspromillen'].astype(float)


print(combined_df['Liggetid'].dtype)
combined_df['Liggetid'] = combined_df['Liggetid'].astype(str)
combined_df['Liggetid'] = combined_df['Liggetid'].str.lstrip('Til salg i alt:').str.rstrip('e').str.rstrip('dag')
combined_df['Liggetid'] = combined_df['Liggetid'].astype(float)


print(combined_df['m2'].dtype)
combined_df['m2'] = combined_df['m2'].astype(str)
combined_df['m2'] = combined_df['m2'].str.rstrip('m²')
combined_df = combined_df.drop(combined_df[combined_df['m2'] == '- '].index)
combined_df['m2'] = combined_df['m2'].astype(float)

#OBS VED MANGLENDE GRUNDSTØRRELSE HAR JEG INDSAT LEJLIGHEDENS AREAL FRA M2
print(combined_df['Grund'].dtype)
combined_df['Grund'] = combined_df['Grund'].astype(str)
combined_df['Grund'] = combined_df['Grund'].str.extract(r'(\d+)').astype(float)
combined_df['Grund'] = combined_df['Grund'].fillna(combined_df['m2'])


print(combined_df['Rooms'].dtype)
combined_df['Rooms'] = combined_df['Rooms'].astype('str')
combined_df['Rooms'] = combined_df['Rooms'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else np.nan)
#Dealing with missing Rooms: (Taking the mean of each "boligtype" and apply that)
mean_Rooms_by_type = combined_df.groupby('Boligtype')['Rooms'].mean()
combined_df['Rooms_imputed'] = combined_df['Rooms'].isna().astype(int) 

def fill_na_Rooms(row):
    if pd.isna(row['Rooms']):
        return mean_Rooms_by_type[row['Boligtype']]
    else:
        return row['Rooms']
# Apply the function to fill NaN values
combined_df['Rooms'] = combined_df.apply(fill_na_Rooms, axis=1)




object
object
object
object
object
object
object


C:\Users\matli\AppData\Local\Temp\ipykernel_75432\63069814.py:43: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_Rooms_by_type = combined_df.groupby('Boligtype')['Rooms'].mean()


In [325]:
#### Price clearning ####

def clean_price(price):
    # Remove the currency symbol and dots, then convert to float
    return float(price.replace(' kr.', '').replace('.', ''))

# Apply the function to the Price column
combined_df.loc[:, 'Price'] = combined_df['Price'].apply(clean_price)


#### Distance to school cleaning ####
def convert_distance(distance):
    if 'km' in distance:
        # Replace comma with a dot and remove the 'km' text
        return float(distance.replace(' km', '').replace(',', '.'))
    elif 'm' in distance:
        # Extract the number, convert to float and divide by 1000 to convert to kilometers
        return float(distance.replace(' m', '').replace(',', '.')) / 1000
    return None

# Apply the function to the 'Distance to school' column
combined_df.loc[:, 'Distance to school'] = combined_df['Distance to school'].apply(convert_distance)


#### Trivsel cleaning ####
combined_df['Trivsel'] = combined_df['Trivsel'].replace(['Ikke oplyst'], pd.NA)

# Convert the 'Trivsel' column to float, handling NaN values appropriately
combined_df['Trivsel'] = pd.to_numeric(combined_df['Trivsel'], errors='coerce')

# Step 1: Calculate the mean 'Trivsel' for each Kommune
trivsel_sort_by='Boligtype'                               #OBS VÆLG KOMMUNE ISTEDET FOR BOLIG TYPE!!!!!!!!!!!!

trivsel_mean_by = combined_df.groupby(trivsel_sort_by)['Trivsel'].mean()
def fill_na_trivsel(row):
    if pd.isna(row['Trivsel']):
        return trivsel_mean_by[row[trivsel_sort_by]]
    else: 
        return row['Trivsel']
# Step 2: Create a dummy column indicating missing 'Trivsel' values (before imputation)
combined_df['Trivsel_imputed'] = combined_df['Trivsel'].isna().astype(int)

# Step 3: Fill missing 'Trivsel' values with the mean for each kommune
combined_df['Trivsel'] = combined_df.apply(fill_na_trivsel, axis=1)

#### GRADES cleaning ####
combined_df['School-grades'] = combined_df['School-grades'].replace(['Ikke oplyst'], pd.NA)

# Convert the 'Trivsel' column to float, handling NaN values appropriately
combined_df['School-grades'] = pd.to_numeric(combined_df['School-grades'], errors='coerce')

# Step 1: Calculate the mean 'Trivsel' for each Kommune
School_grades_sort_by='Boligtype'                               #OBS VÆLG KOMMUNE ISTEDET FOR BOLIGTYPE!!!!!!!!!!!!


School_grades_mean_by = combined_df.groupby(School_grades_sort_by)['School-grades'].mean()


def fill_na_grades(row):
    if pd.isna(row['School-grades']):
        return School_grades_mean_by[row[School_grades_sort_by]]
    else: 
        return row['School-grades']


# Step 2: Create a dummy column indicating missing 'grades' values (before imputation)
combined_df['Grades_imputed'] = combined_df['School-grades'].isna().astype(int)

# Step 3: Fill missing 'grades' values with the mean for each kommune
combined_df['School-grades'] = combined_df.apply(fill_na_grades, axis=1)


#Price
combined_df['Price'] = combined_df['Price'].astype('float')
#Distance to school
combined_df['Distance to school'] = combined_df['Distance to school'].astype('float')


C:\Users\matli\AppData\Local\Temp\ipykernel_75432\965844473.py:34: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  trivsel_mean_by = combined_df.groupby(trivsel_sort_by)['Trivsel'].mean()
C:\Users\matli\AppData\Local\Temp\ipykernel_75432\965844473.py:56: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  School_grades_mean_by = combined_df.groupby(School_grades_sort_by)['School-grades'].mean()



### Her starter databehandlingen


In [327]:
####BUILD
#combined_df = combined_df.replace('fejl', np.NaN)
combined_df['Build'] = combined_df['Build'].astype('str')
# Extract numbers from the 'Build' column using regex
combined_df['Build'] = combined_df['Build'].apply(lambda x: re.findall(r'\d+', x)[0] if re.findall(r'\d+', x) else np.nan)
# Making it intergers:
combined_df['Build'] = combined_df['Build'].astype('int')

###Toilets
combined_df['Toilets'] = combined_df['Toilets'].astype('str')
# Extract numbers from the 'Toilets' column using regex.. Setting missing values to NaN
combined_df['Toilets'] = combined_df['Toilets'].apply(lambda x: int(re.findall(r'\d+', x)[0]) if re.findall(r'\d+', x) else np.nan)
#Dealing with missing toilets: (Taking the mean of each "boligtype" and apply that)
mean_toilets_by_type = combined_df.groupby('Boligtype')['Toilets'].mean()
combined_df['Toilets_imputed'] = combined_df['Toilets'].isna().astype(int) 

def fill_na_toilets(row):
    if pd.isna(row['Toilets']):
        return mean_toilets_by_type[row['Boligtype']]
    else:
        return row['Toilets']
# Apply the function to fill NaN values
combined_df['Toilets'] = combined_df.apply(fill_na_toilets, axis=1)

###Elevator -> Setting values to 0 if either NaN or Elevator: Nej
combined_df['Elevator'] = combined_df['Elevator'].astype('str')
combined_df['Elevator'] = combined_df['Elevator'].apply(lambda x: 1 if re.findall(r'\bJa\b', x) else 0)

###Altan
combined_df['Altan'] = combined_df['Altan'].astype('str')
combined_df['Altan'] = combined_df['Altan'].apply(lambda x: 1 if re.findall(r'\bJa\b', x) else 0)

###Terasse
combined_df['Terasse'] = combined_df['Terasse'].astype('str')
combined_df['Terasse'] = combined_df['Terasse'].apply(lambda x: 1 if re.findall(r'\bJa\b', x) else 0)

###Dropping Toilet-column
combined_df.drop('Toilet', axis=1, inplace=True)

###Postial code
combined_df['Postal code'] = combined_df['Postal code'].astype('category')

###Energy-classification
combined_df['Energy-classification'] = combined_df['Energy-classification'].astype('str')
combined_df['Energy-classification'] = combined_df['Energy-classification'].str.lstrip('Energimærke')
combined_df['Energy-classification'] = combined_df['Energy-classification'].astype('category')

###Radon
combined_df['Radon'] = combined_df['Radon'].astype('category')




C:\Users\matli\AppData\Local\Temp\ipykernel_75432\4196732420.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mean_toilets_by_type = combined_df.groupby('Boligtype')['Toilets'].mean()


In [216]:
combined_df
combined_df[pd.isna(combined_df['Toilets'])]
combined_df[combined_df['Elevator'] == 1]

#combined_df['Boligtype'].astype('str').unique()
combined_df
combined_df[combined_df['Toilets_imputed'] == 1]
combined_df
combined_df['Boligtype'].astype('str').unique()
combined_df[combined_df['Boligtype'] == 'Villalejlighed']

,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Build,Toilets,Postal code,link,Elevator,Altan,Terasse,Trivsel_imputed,Grades_imputed,Toilets_imputed
"Dirchsvej 34, 1.,2300",5195000.0,Gerbrandskolen,0.66,6.7,3.8,Villalejlighed,D,Mellem,23.7,34.0,...,1943,1.0,2300,https://www.boligsiden.dk/addresses/0a3f509d-c94a-32b8-e044-0003ba298018,0,0,0,0,0,0


In [301]:
combined_df['Boligtype'].astype('str').unique()

array(['Rækkehus', 'Ejerlejlighed', 'Landejendom', 'Villa',
       'Villalejlighed'], dtype=object)

In [329]:
#Duplicates
combined_df = combined_df.drop_duplicates()
combined_df

,Price,Schoolname,Distance to school,School-grades,Trivsel,Boligtype,Energy-classification,Radon,Kommuneskat,Grundskyldspromillen,...,Toilets,Postal code,link,Elevator,Altan,Terasse,Rooms_imputed,Trivsel_imputed,Grades_imputed,Toilets_imputed
"Mellemtoftevej 36,2600",3995000.0,"Hospitalsskolen , Børne og Ungdomspsykiatrisk Center",0.391,7.187918,3.728406,Rækkehus,D,Høj,24.60,25.00,...,1.0,2600,https://www.boligsiden.dk/addresses/0a3f50a4-be85-32b8-e044-0003ba298018,0,0,0,0,1,1,0
"Strandpromenaden 21, 1. tv.,3000",6095000.0,Skolen i Bymidten,0.336,7.182988,3.723773,Ejerlejlighed,A2015,Lav,25.52,28.50,...,2.0,3000,https://www.boligsiden.dk/addresses/0d5cec39-0046-4f36-9850-8c2d08933fd5,0,0,0,0,1,1,0
"Hjortdalvej 331,9460",3995000.0,Skovsgård Tranum skole,5.240,6.900000,3.600000,Landejendom,D,Meget lav,25.70,32.85,...,1.0,9460,https://www.boligsiden.dk/addresses/39ff92cd-f2b1-0991-e044-0003ba298018,0,0,0,0,0,0,0
"Lystgårdsparken 34,8300",2798000.0,Parkvejens Skole,5.160,7.000000,3.700000,Villa,B,Mellem,25.10,30.32,...,1.0,8300,https://www.boligsiden.dk/addresses/363cc2db-6e07-1eb8-e044-0003ba298018,0,0,0,0,0,0,0
"Fuglsøvej 92,8420",6000000.0,Molsskolen,3.740,7.400000,3.700000,Villa,D,Høj,25.90,33.80,...,4.0,8420,https://www.boligsiden.dk/addresses/0a3f50bf-6f17-32b8-e044-0003ba298018,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Langelandsvej 23,6100",1145000.0,Starup-Øsby Skole og Børnehus,1.140,6.400000,3.800000,Villa,D,Høj,26.30,24.36,...,2.0,6100,https://www.boligsiden.dk/addresses/0a3f50b6-c77b-32b8-e044-0003ba298018,0,0,0,0,0,0,0
"Nybovej 7,9640",995000.0,Farsø Skole,0.373,6.900000,3.700000,Villa,D,Mellem,26.30,22.82,...,1.0,9640,https://www.boligsiden.dk/addresses/0a3f50c7-2cfc-32b8-e044-0003ba298018,0,0,0,0,0,0,0
"Storskovvej 3,4700",2995000.0,"Fladsåskolen, afd. Korskilde",4.910,7.013961,3.717343,Villa,A2010,Høj,25.00,25.00,...,1.0,4700,https://www.boligsiden.dk/addresses/0a3f50af-9285-32b8-e044-0003ba298018,0,0,0,0,1,1,0
"Gl Landevej 23,9690",725000.0,Fjerritslev Skole,5.620,6.400000,3.600000,Villa,E,Lav,25.70,32.85,...,1.0,9690,https://www.boligsiden.dk/addresses/0a3f50c7-3af4-32b8-e044-0003ba298018,0,0,0,0,0,0,0


In [399]:
#Retrieving muncipialities
"""print(combined_df['Kommune tekst'])"""
combined_df['Muncipiality'] = combined_df['Kommune tekst'].apply(lambda x: re.findall(r'indbyggere i (.*?)\.', x)[0])
combined_df.drop('Kommune tekst', axis=1, inplace=True) #Removing the "kommune tekst" column. 
print(combined_df.columns)

Index(['Price', 'Schoolname', 'Distance to school', 'School-grades', 'Trivsel',
       'Boligtype', 'Energy-classification', 'Radon', 'Kommuneskat',
       'Grundskyldspromillen', 'Liggetid', 'm2', 'Grund', 'Ejerudgift',
       'Rooms', 'Build', 'Toilets', 'Postal code', 'link', 'Elevator', 'Altan',
       'Terasse', 'Rooms_imputed', 'Trivsel_imputed', 'Grades_imputed',
       'Toilets_imputed', 'Muncipiality'],
      dtype='object')


In [401]:
combined_df.to_csv(f'outputdata/df_final.csv', index=True)